In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('historical_data.csv')

In [3]:
df['date_txn']=pd.to_datetime(df.date_txn)   #converting the datatype of date_txn to datetime inorder to perform some datetime operations


In [4]:
df['weekday']=df.date_txn.dt.weekday  #to know which day of the week it is
                                     #(there might be some trends of the variation of stock price with respect to the weekdays)
df['week']=df.date_txn.dt.week

In [5]:
df

,symbol,date_txn,close_price,predicted_price,weekday,week
0,AAL,2014-12-02,47.55,47.93,1,49
1,AAL,2014-12-03,47.93,49.33,2,49
2,AAL,2014-12-04,49.33,50.68,3,49
3,AAL,2014-12-05,50.68,50.50,4,49
4,AAL,2014-12-08,50.50,48.00,0,50
...,...,...,...,...,...,...
1048570,KMF,2015-08-12,22.76,21.94,2,33
1048571,KMF,2015-08-13,21.94,22.51,3,33
1048572,KMF,2015-08-14,22.51,22.30,4,33
1048573,KMF,2015-08-17,22.30,22.10,0,34


In [6]:
df.isnull().sum()

symbol               0
date_txn             0
close_price          0
predicted_price    777
weekday              0
week                 0
dtype: int64

In [7]:
df.fillna(0, inplace=True)  #filling the missing values with 0
for k in range(len(df)):    #replacing the 0 values(predicted_price) with the close price of the next row.
    if df.iloc[k,3]==0:     #because we have to predict the closing price of the next day and the closing prices are non-empty
        
        df.iloc[k,3]= df.iloc[k+1,2]
    

In [8]:
dup_df=df.copy() #making a copy

In [9]:
dup_df['yest_cp']=dup_df[['close_price']].shift(+1) #shifting by 1 row and creating a new column 
dup_df['daybyest_cp']=dup_df[['close_price']].shift(+2) #because the closing price of the previous day can be used as a feature for training our model 

In [10]:
dup_df   #two new columns are introduced : yesterday's clocsing price(yest_cp) and the day before yest CP(daybyest_cp)

,symbol,date_txn,close_price,predicted_price,weekday,week,yest_cp,daybyest_cp
0,AAL,2014-12-02,47.55,47.93,1,49,NaN,NaN
1,AAL,2014-12-03,47.93,49.33,2,49,47.55,NaN
2,AAL,2014-12-04,49.33,50.68,3,49,47.93,47.55
3,AAL,2014-12-05,50.68,50.50,4,49,49.33,47.93
4,AAL,2014-12-08,50.50,48.00,0,50,50.68,49.33
...,...,...,...,...,...,...,...,...
1048570,KMF,2015-08-12,22.76,21.94,2,33,22.06,22.20
1048571,KMF,2015-08-13,21.94,22.51,3,33,22.76,22.06
1048572,KMF,2015-08-14,22.51,22.30,4,33,21.94,22.76
1048573,KMF,2015-08-17,22.30,22.10,0,34,22.51,21.94


In [11]:
prev_week=[]    #creating empty lists
prev_week_cp=[]  #prev_week_cp will contain the closing price of the same weekday as in the previous week
list1=[]         #list1 will contain the indices for which we want the closing price of the previous weekday.
list2=[]         #list1 and prev_week_cp contain corresponding values
list3=[]         #the given dataset consists of many segments with several symbols and their closing prices are not inter-related.
                 #so for every symbol we have to perform the same operations and the following block of code does that
                 
for i in range(len(dup_df)):
    try:
        if dup_df.iloc[i,0]==dup_df.iloc[i+1,0]:   #it proceeds only if the two subsequent symbols are same
            j=0
            while (dup_df.iloc[i,4]!=dup_df.iloc[i+j+1,4]) :  #this block checks for the weekdays and proceeds only if they are unequal
                j+=1
                if (dup_df.iloc[i,4]==dup_df.iloc[i+j+1,4]) & (dup_df.iloc[i+j+1,0]==dup_df.iloc[i,0]): #it checks if the two considered weekdays are same and the corresponding symbols are same or not
                    list1.append(i+j+1)                                 #because there will be situation when the considered weekdays will be same but the later weekday will be of the next symbol
                    prev_week_cp.append(dup_df.iloc[i,2])               #and we want to avoid such situation
                    break
                                   
                    
                    
            else:                                           #this block considers when the two subsequent weekdays are equal
                if (dup_df.iloc[i,4]==dup_df.iloc[i+j+1,4]) & (dup_df.iloc[i+j+1,0]==dup_df.iloc[i,0]):
                    prev_week_cp.append(dup_df.iloc[i,2])
                    prev_week.append(dup_df.iloc[i,2])
                    list1.append(i+j+1)
                   
        else:
            list2.append(i)       #this list contains the last rows of every symbol
            #print('breaking : ',i)
                    
    except Exception as e:
        list3.append(i)
        print('last line ', i)
        pass

last line  1048570
last line  1048571
last line  1048572
last line  1048573
last line  1048574


In [12]:
print(len(list1))   #checking whether the lists are of the same length or not
print(len(prev_week_cp))

1044650
1044650


In [13]:
1044650+3925  #785*5=3925

1048575

In [14]:
len(list2)

784

In [15]:
new_df = pd.DataFrame(list1, columns=['index']) #creating a new dataframe out of the list i,e list1

In [16]:
new_df

,index
0,5
1,6
2,7
3,8
4,9
...,...
1044645,1048570
1044646,1048571
1044647,1048572
1044648,1048573


In [17]:
new_df['prev_week_cp'] = pd.DataFrame(prev_week_cp)

In [18]:
new_df

,index,prev_week_cp
0,5,47.55
1,6,47.93
2,7,49.33
3,8,50.68
4,9,50.50
...,...,...
1044645,1048570,23.02
1044646,1048571,22.27
1044647,1048572,21.90
1044648,1048573,22.20


In [19]:
new_df.sort_values(by='index', ascending=True)  #rearranging the indices in ascending order
                                                #because at some places the indices are not in order

,index,prev_week_cp
0,5,47.55
1,6,47.93
2,7,49.33
3,8,50.68
4,9,50.50
...,...,...
1044645,1048570,23.02
1044646,1048571,22.27
1044647,1048572,21.90
1044648,1048573,22.20


In [20]:
new_df2=new_df.set_index('index')  #setting the index columnn as index

In [21]:
new_df2

,prev_week_cp
index,
5,47.55
6,47.93
7,49.33
8,50.68
9,50.50
...,...
1048570,23.02
1048571,22.27
1048572,21.90


In [22]:
df_concat = pd.concat([dup_df, new_df2], axis=1, sort=False) #concatenating the two dataframes
                                                             #the new dataframe is the union of both

In [23]:
df_concat

,symbol,date_txn,close_price,predicted_price,weekday,week,yest_cp,daybyest_cp,prev_week_cp
0,AAL,2014-12-02,47.55,47.93,1,49,NaN,NaN,NaN
1,AAL,2014-12-03,47.93,49.33,2,49,47.55,NaN,NaN
2,AAL,2014-12-04,49.33,50.68,3,49,47.93,47.55,NaN
3,AAL,2014-12-05,50.68,50.50,4,49,49.33,47.93,NaN
4,AAL,2014-12-08,50.50,48.00,0,50,50.68,49.33,NaN
...,...,...,...,...,...,...,...,...,...
1048570,KMF,2015-08-12,22.76,21.94,2,33,22.06,22.20,23.02
1048571,KMF,2015-08-13,21.94,22.51,3,33,22.76,22.06,22.27
1048572,KMF,2015-08-14,22.51,22.30,4,33,21.94,22.76,21.90
1048573,KMF,2015-08-17,22.30,22.10,0,34,22.51,21.94,22.20


In [24]:
df_concat['last_wd_cp']=df_concat[['prev_week_cp']].shift(-1) #shifting by one row above, because we are to predict for the next day.
df_concat['next_wd']=df_concat[['weekday']].shift(-1)         #and we have to consider the close price of the last weekday as of the next day
                                                              #similarly shifting the weekday column

In [28]:
df_concat

,symbol,date_txn,close_price,predicted_price,weekday,week,yest_cp,daybyest_cp,prev_week_cp,last_wd_cp,next_wd
0,AAL,2014-12-02,47.55,47.93,1,49,NaN,NaN,NaN,NaN,2.0
1,AAL,2014-12-03,47.93,49.33,2,49,47.55,NaN,NaN,NaN,3.0
2,AAL,2014-12-04,49.33,50.68,3,49,47.93,47.55,NaN,NaN,4.0
3,AAL,2014-12-05,50.68,50.50,4,49,49.33,47.93,NaN,NaN,0.0
4,AAL,2014-12-08,50.50,48.00,0,50,50.68,49.33,NaN,47.55,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1048570,KMF,2015-08-12,22.76,21.94,2,33,22.06,22.20,23.02,22.27,3.0
1048571,KMF,2015-08-13,21.94,22.51,3,33,22.76,22.06,22.27,21.90,4.0
1048572,KMF,2015-08-14,22.51,22.30,4,33,21.94,22.76,21.90,22.20,0.0
1048573,KMF,2015-08-17,22.30,22.10,0,34,22.51,21.94,22.20,22.06,1.0


In [25]:
df_concat.isnull().sum()

symbol                0
date_txn              0
close_price           0
predicted_price       0
weekday               0
week                  0
yest_cp               1
daybyest_cp           2
prev_week_cp       3925
last_wd_cp         3925
next_wd               1
dtype: int64

In [26]:
df_concat.dropna(inplace=True) #dropping the rows containing NaN or missing values

In [27]:
df_concat.isnull().sum()

symbol             0
date_txn           0
close_price        0
predicted_price    0
weekday            0
week               0
yest_cp            0
daybyest_cp        0
prev_week_cp       0
last_wd_cp         0
next_wd            0
dtype: int64

In [28]:
df_concat

,symbol,date_txn,close_price,predicted_price,weekday,week,yest_cp,daybyest_cp,prev_week_cp,last_wd_cp,next_wd
5,AAL,2014-12-09,48.00,48.63,1,50,50.50,50.68,47.55,47.93,2.0
6,AAL,2014-12-10,48.63,50.21,2,50,48.00,50.50,47.93,49.33,3.0
7,AAL,2014-12-11,50.21,49.65,3,50,48.63,48.00,49.33,50.68,4.0
8,AAL,2014-12-12,49.65,50.52,4,50,50.21,48.63,50.68,50.50,0.0
9,AAL,2014-12-15,50.52,47.65,0,51,49.65,50.21,50.50,48.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1048569,KMF,2015-08-11,22.06,22.76,1,33,22.20,21.90,24.23,23.02,2.0
1048570,KMF,2015-08-12,22.76,21.94,2,33,22.06,22.20,23.02,22.27,3.0
1048571,KMF,2015-08-13,21.94,22.51,3,33,22.76,22.06,22.27,21.90,4.0
1048572,KMF,2015-08-14,22.51,22.30,4,33,21.94,22.76,21.90,22.20,0.0


In [29]:
df_concat.to_csv('cleaned_tinoIQ2.csv', index=False)

In [30]:
df_concat = pd.read_csv('cleaned_tinoIQ2.csv')

In [31]:
df_concat

,symbol,date_txn,close_price,predicted_price,weekday,week,yest_cp,daybyest_cp,prev_week_cp,last_wd_cp,next_wd
0,AAL,2014-12-09,48.00,48.63,1,50,50.50,50.68,47.55,47.93,2.0
1,AAL,2014-12-10,48.63,50.21,2,50,48.00,50.50,47.93,49.33,3.0
2,AAL,2014-12-11,50.21,49.65,3,50,48.63,48.00,49.33,50.68,4.0
3,AAL,2014-12-12,49.65,50.52,4,50,50.21,48.63,50.68,50.50,0.0
4,AAL,2014-12-15,50.52,47.65,0,51,49.65,50.21,50.50,48.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1043851,KMF,2015-08-11,22.06,22.76,1,33,22.20,21.90,24.23,23.02,2.0
1043852,KMF,2015-08-12,22.76,21.94,2,33,22.06,22.20,23.02,22.27,3.0
1043853,KMF,2015-08-13,21.94,22.51,3,33,22.76,22.06,22.27,21.90,4.0
1043854,KMF,2015-08-14,22.51,22.30,4,33,21.94,22.76,21.90,22.20,0.0


In [32]:
date_list=[df_concat.iloc[i,1].split('-') for i in range(len(df_concat))]   #converting date_txn to string format without '-'
dates = [''.join(i) for i in date_list]

In [33]:
type(dates[0])

str

In [34]:
df_concat['Date_txn'] = pd.DataFrame(dates)  #adding dates column and concatenating it to the df_concat dataframe

In [35]:
df_concat

,symbol,date_txn,close_price,predicted_price,weekday,week,yest_cp,daybyest_cp,prev_week_cp,last_wd_cp,next_wd,Date_txn
0,AAL,2014-12-09,48.00,48.63,1,50,50.50,50.68,47.55,47.93,2.0,20141209
1,AAL,2014-12-10,48.63,50.21,2,50,48.00,50.50,47.93,49.33,3.0,20141210
2,AAL,2014-12-11,50.21,49.65,3,50,48.63,48.00,49.33,50.68,4.0,20141211
3,AAL,2014-12-12,49.65,50.52,4,50,50.21,48.63,50.68,50.50,0.0,20141212
4,AAL,2014-12-15,50.52,47.65,0,51,49.65,50.21,50.50,48.00,1.0,20141215
...,...,...,...,...,...,...,...,...,...,...,...,...
1043851,KMF,2015-08-11,22.06,22.76,1,33,22.20,21.90,24.23,23.02,2.0,20150811
1043852,KMF,2015-08-12,22.76,21.94,2,33,22.06,22.20,23.02,22.27,3.0,20150812
1043853,KMF,2015-08-13,21.94,22.51,3,33,22.76,22.06,22.27,21.90,4.0,20150813
1043854,KMF,2015-08-14,22.51,22.30,4,33,21.94,22.76,21.90,22.20,0.0,20150814


In [36]:
feature_set = df_concat.drop(columns=['date_txn','weekday','week','prev_week_cp','predicted_price'])  #this is our independent variable X
target_set = df_concat.iloc[:,3]     #this is our dependent/target variable

In [37]:
feature_set

,symbol,close_price,yest_cp,daybyest_cp,last_wd_cp,next_wd,Date_txn
0,AAL,48.00,50.50,50.68,47.93,2.0,20141209
1,AAL,48.63,48.00,50.50,49.33,3.0,20141210
2,AAL,50.21,48.63,48.00,50.68,4.0,20141211
3,AAL,49.65,50.21,48.63,50.50,0.0,20141212
4,AAL,50.52,49.65,50.21,48.00,1.0,20141215
...,...,...,...,...,...,...,...
1043851,KMF,22.06,22.20,21.90,23.02,2.0,20150811
1043852,KMF,22.76,22.06,22.20,22.27,3.0,20150812
1043853,KMF,21.94,22.76,22.06,21.90,4.0,20150813
1043854,KMF,22.51,21.94,22.76,22.20,0.0,20150814


In [38]:
target_set

0          48.63
1          50.21
2          49.65
3          50.52
4          47.65
           ...  
1043851    22.76
1043852    21.94
1043853    22.51
1043854    22.30
1043855    22.10
Name: predicted_price, Length: 1043856, dtype: float64

In [39]:
X = df_concat.drop(columns=['date_txn','weekday','week','prev_week_cp','predicted_price'])
y = df_concat.iloc[:,3]               #Extracting our independent and dependent variables for training our Linear Regression model

In [40]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder=LabelEncoder()
X.iloc[:,0] = labelencoder.fit_transform(X.iloc[:,0])  #transforming the symbol variables to numerical values
onehotencoder=OneHotEncoder(categorical_features=[0])
X = onehotencoder.fit_transform(X).toarray()

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [41]:
from sklearn.model_selection import train_test_split           #splitting the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.3, random_state=0)

In [42]:
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(X_train, y_train)         #fitting our training set i.e training our model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [43]:
y_pred = regressor.predict(X_test)    #predicting our testing set using our trained model

In [44]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)              #we see high r2 score meaning our model well fits our dataset

0.9987490497275066

In [45]:
y_pred

array([  1.764995  ,  24.27009192,  11.04451491, ..., 139.59347722,
       154.48332217,  24.97720462])

In [46]:
y_test

675650      1.73
622166     24.71
45895      11.00
840575    266.77
467450     13.20
           ...  
369539      4.36
23256      26.90
603136    140.12
431644    158.28
361274     24.87
Name: predicted_price, Length: 313157, dtype: float64

In [47]:
prediction=regressor.predict(X)       #predicting the whole datset 

In [48]:
final_df=df_concat.copy()

In [49]:
final_df.drop(columns=['Date_txn','weekday','week','prev_week_cp'], inplace=True)

In [50]:
final_df['pred_col']=pd.DataFrame(prediction)     #inserting our prediction into our final dataframe to compare

In [51]:
final_df

,symbol,date_txn,close_price,predicted_price,yest_cp,daybyest_cp,last_wd_cp,next_wd,pred_col
0,AAL,2014-12-09,48.00,48.63,50.50,50.68,47.93,2.0,48.623640
1,AAL,2014-12-10,48.63,50.21,48.00,50.50,49.33,3.0,48.583573
2,AAL,2014-12-11,50.21,49.65,48.63,48.00,50.68,4.0,49.687156
3,AAL,2014-12-12,49.65,50.52,50.21,48.63,50.50,0.0,49.600315
4,AAL,2014-12-15,50.52,47.65,49.65,50.21,48.00,1.0,50.185314
...,...,...,...,...,...,...,...,...,...
1043851,KMF,2015-08-11,22.06,22.76,22.20,21.90,23.02,2.0,22.043618
1043852,KMF,2015-08-12,22.76,21.94,22.06,22.20,22.27,3.0,22.532219
1043853,KMF,2015-08-13,21.94,22.51,22.76,22.06,21.90,4.0,22.093052
1043854,KMF,2015-08-14,22.51,22.30,21.94,22.76,22.20,0.0,22.328732


In [52]:
#creating a median column according to the given problem
#we have taken the absolute value because negative values also are generated
#and while taking the average this would give unwanted results
#given in the problem set X = Median of  (predicted_price-close_price)/close_price
#the pred_col is our predicted_price and the predicted_price is the close_price which is to be predicted
final_df['median']=abs((final_df['pred_col']-final_df['predicted_price']))/final_df['predicted_price'] 

In [53]:
from statistics import median
median_df=final_df.groupby(['symbol']).median()    #grouping by the symbols and taking the median value

In [54]:
median_df.mean(axis=0)    #taking the average of all the median values for every symbol
                      

close_price        58.606873
predicted_price    58.621242
yest_cp            58.590854
daybyest_cp        58.574548
last_wd_cp         58.543809
next_wd             2.000637
pred_col           58.627634
median              0.011363
dtype: float64

Thus, the average is found to be 0.011363